In [1]:
%cd ../..
%load_ext autoreload
%autoreload 2

/home/jupyter/map-ephys


In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from pipeline import lab, get_schema_name, experiment, foraging_model, ephys, foraging_analysis, histology, ccf, report, psth_foraging
from pipeline.plot import behavior_plot, unit_characteristic_plot, unit_psth, histology_plot, PhotostimError, foraging_plot
from pipeline.plot.foraging_model_plot import plot_session_model_comparison, plot_session_fitted_choice, plot_unit_all_in_one

foraging_sessions = (foraging_analysis.SessionTaskProtocol & 'session_task_protocol=100').proj()
import datajoint as dj; dj.conn().connect()

[2022-11-22 20:51:06,751][INFO]: Connecting map-superuser@datajoint.mesoscale-activity-map.org:3306
[2022-11-22 20:51:06,798][INFO]: Connected map-superuser@datajoint.mesoscale-activity-map.org:3306


# Export cache tables for streamlit app

## All foraging sessions

In [63]:
def fetch_sessions():
    # with st.spinner(f'Connect to datajoint...'):
    
    # with st.spinner(f'Fetching...'):
    foraging_sessions = (foraging_analysis.SessionTaskProtocol & 'session_task_protocol=100').proj()
    insertion_numbers = foraging_sessions.aggr(foraging_sessions * ephys.ProbeInsertion, ..., 
                                                    #   keep_all_rows=True, ephys_insertions='IF(COUNT(insertion_number), "yes", "no")')
                                                keep_all_rows=True, ephys_ins='COUNT(insertion_number)')
    if_histology = foraging_sessions.aggr(foraging_sessions * histology.ElectrodeCCFPosition.ElectrodePosition, ...,
                                          keep_all_rows=True, histology='IF(COUNT(ccf_x)>0, "yes", "no")')
    if_photostim_from_behav = foraging_sessions.aggr(foraging_sessions * experiment.PhotostimForagingTrial, ...,
                                          keep_all_rows=True, photostim_behav='IF(COUNT(trial)>0, "yes", "no")')
    if_photostim_from_ephys = foraging_sessions.aggr(foraging_sessions * (ephys.TrialEvent & 'trial_event_type LIKE "laser%"'), ...,
                                          keep_all_rows=True, photostim_NI='IF(COUNT(trial)>0, "yes", "no")')

    df_sessions = pd.DataFrame(((experiment.Session & foraging_sessions)
                                * lab.WaterRestriction.proj(h2o='water_restriction_number')
                                * insertion_numbers
                                * if_histology
                                * if_photostim_from_behav
                                * if_photostim_from_ephys)
                               .proj(..., '-rig', '-username', '-session_time')
                               .fetch()
                                )
    # df_sessions['session_date'] = pd.to_datetime(df_sessions['session_date'], format="%Y-%m-%d")
    df_sessions.to_pickle(cache_folder + 'sessions.pkl')
    
    return df_sessions

fetch_sessions()

,subject_id,session,session_date,h2o,ephys_ins,histology,photostim_behav,photostim_NI
0,447921,1,2019-09-11,HC16,0,no,no,no
1,447921,2,2019-09-12,HC16,0,no,no,no
2,447921,3,2019-09-13,HC16,0,no,no,no
3,447921,4,2019-09-14,HC16,0,no,no,no
4,447921,5,2019-09-15,HC16,0,no,no,no
...,...,...,...,...,...,...,...,...
1792,634898,15,2022-09-02,KH_FB39,0,no,no,no
1793,634898,16,2022-09-06,KH_FB39,0,no,no,no
1794,634898,17,2022-09-07,KH_FB39,0,no,no,no
1795,634898,18,2022-09-08,KH_FB39,0,no,no,no


## Ephys units

In [55]:
cache_folder = '~/s3-drive/report/st_cache/'

# def fetch_ephys_units():
# df = pd.DataFrame(all_unit_qc.fetch())

t_iti_delta_Q = (psth_foraging.UnitPeriodLinearFit.Param & 'period in ("iti_all")' & 'multi_linear_model = "Q_rel + Q_tot + rpe"' 
& 'var_name = "relative_action_value_ic"').proj(dQ_iti='t', dQ_iti_abs='ABS(t)', _1='period', __1='var_name')

t_trial_delta_Q = (psth_foraging.UnitPeriodLinearFit.Param & 'period in ("go_to_end")' & 'multi_linear_model = "Q_rel + Q_tot + rpe"' 
& 'var_name = "relative_action_value_ic"').proj(dQ_trial='t', dQ_trial_abs='ABS(t)', _2='period', __2='var_name')

t_iti_sum_Q = (psth_foraging.UnitPeriodLinearFit.Param & 'period in ("iti_all")' & 'multi_linear_model = "Q_rel + Q_tot + rpe"' 
& 'var_name = "total_action_value"').proj(sumQ_iti='t', sumQ_iti_abs='ABS(t)', _3='period', __3='var_name')

t_trial_sum_Q = (psth_foraging.UnitPeriodLinearFit.Param & 'period in ("go_to_end")' & 'multi_linear_model = "Q_rel + Q_tot + rpe"' 
& 'var_name = "total_action_value"').proj(sumQ_trial='t', sumQ_trial_abs='ABS(t)', _4='period', __4='var_name')

t_iti_rpe = (psth_foraging.UnitPeriodLinearFit.Param & 'period in ("iti_all")' & 'multi_linear_model = "Q_rel + Q_tot + rpe"' 
& 'var_name = "rpe"').proj(rpe_iti='t', rpe_iti_abs='ABS(t)', _5='period', __5='var_name')

t_trial_rpe = (psth_foraging.UnitPeriodLinearFit.Param & 'period in ("go_to_end")' & 'multi_linear_model = "Q_rel + Q_tot + rpe"' 
& 'var_name = "rpe"').proj(rpe_trial='t', rpe_trial_abs='ABS(t)', _6='period', __6='var_name')

q_ccf = (histology.ElectrodeCCFPosition.ElectrodePosition * ccf.CCFAnnotation).proj()

q_all_unit = (report.UnitLevelForagingEphysReportAllInOne.key_source  # With behavior QC and miminal unit QC
                * ephys.Unit.proj('unit_uid', 'unit_amp', 'unit_snr', 'electrode')
                * lab.WaterRestriction.proj(h2o='water_restriction_number')
                * experiment.Session.proj(session_date="cast(concat(session_date) as datetime)")
                # * ephys.ClusterMetric.proj('presence_ratio', 'amplitude_cutoff')
                # * ephys.UnitStat
                * q_ccf
                * t_iti_delta_Q * t_trial_delta_Q * t_iti_sum_Q * t_trial_sum_Q * t_iti_rpe * t_trial_rpe & 'sumQ_iti IS NOT NULL')

df_AOI = ccf.AreaOfInterest.CCFBrainRegionIncluded.proj(annotation='region_name').fetch(format='frame')
df_all_unit = pd.DataFrame(q_all_unit.fetch())

df_all_unit = df_all_unit.merge(df_AOI.reset_index().drop('id', axis=1), how='left', on='annotation').fillna('others')  # Add area of interest
df_all_unit.drop(['clustering_method', 'behavior_model', 'multi_linear_model', 
                 'probe_type', 'electrode_config_name', 'electrode_group', 'ccf_label_id', 'annotation_version', 'unit_uid']
                 + [tmp for tmp in df_all_unit.keys() if tmp[0]=='_'], axis=1, inplace=True)
df_all_unit.rename(columns={'insertion_number': 'ins'}, inplace=True)
df_all_unit = df_all_unit.round(3)

df_all_unit.to_pickle(cache_folder + 'ephys_units.pkl')
   
    # return df_all_unit

#df_all_unit = fetch_ephys_units()

In [68]:
cache_folder = '~/s3-drive/report/st_cache/'
df_all_unit.to_pickle(cache_folder + 'ephys_units.pkl')

In [67]:
df_all_unit = df_all_unit.round(3)
df_all_unit

,subject_id,session,ins,unit,electrode,ccf_x,ccf_y,ccf_z,annotation,unit_amp,...,dQ_trial_abs,sumQ_iti,sumQ_iti_abs,sumQ_trial,sumQ_trial_abs,rpe_iti,rpe_iti_abs,rpe_trial,rpe_trial_abs,area_of_interest
0,473360,51,2,1,2,6080,5000,4360,"Lateral septal nucleus, rostral (rostroventral...",174.728,...,1.853,-6.202,6.202,-5.821,5.821,2.682,2.682,2.942,2.942,LSN
1,473360,51,2,6,6,6080,4940,4340,"Lateral septal nucleus, rostral (rostroventral...",288.157,...,0.294,6.214,6.214,3.667,3.667,2.911,2.911,-1.623,1.623,LSN
2,473360,50,1,3,8,5560,5780,4820,Medial septal nucleus,86.709,...,1.991,-1.741,1.741,-0.582,0.582,-2.486,2.486,-2.210,2.210,others
3,473360,51,2,209,8,6080,4920,4320,"Lateral septal nucleus, rostral (rostroventral...",81.831,...,1.814,3.875,3.875,2.728,2.728,4.021,4.021,5.248,5.248,LSN
4,473360,49,2,13,9,5340,5900,5100,Medial preoptic area,77.287,...,2.520,2.126,2.126,0.331,0.331,-2.583,2.583,-3.579,3.579,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4582,494691,18,3,69,304,6680,2100,3460,"Secondary motor area, layer 5",93.640,...,7.198,-2.971,2.971,-6.513,6.513,-0.654,0.654,3.687,3.687,ALM
4583,494691,18,3,72,308,6680,2060,3460,"Secondary motor area, layer 5",73.835,...,1.971,3.650,3.650,2.080,2.080,1.728,1.728,3.313,3.313,ALM
4584,494691,18,3,80,320,6680,1960,3420,"Secondary motor area, layer 5",191.754,...,0.054,3.213,3.213,2.561,2.561,4.507,4.507,10.545,10.545,ALM
4585,494691,18,3,90,346,6720,1740,3360,"Secondary motor area, layer 2/3",98.785,...,5.546,7.557,7.557,6.210,6.210,2.951,2.951,0.045,0.045,ALM


# Miscs

## Add `AreaOfInterest` lookup table

In [ ]:
ccf.AreaOfInterest.drop()

In [7]:
ccf.AreaOfInterest.CCFBrainRegionIncluded()

area_of_interest,id,annotation_version e.g. CCF_2017,region_name
OLF,0,CCF_2017,Accessory olfactory bulb
OLF,1,CCF_2017,"Accessory olfactory bulb, glomerular layer"
OLF,2,CCF_2017,"Accessory olfactory bulb, granular layer"
OLF,3,CCF_2017,"Accessory olfactory bulb, mitral layer"
OLF,4,CCF_2017,accessory olfactory tract
ACA,0,CCF_2017,Anterior cingulate area
ACA,1,CCF_2017,"Anterior cingulate area, dorsal part"
ACA,2,CCF_2017,"Anterior cingulate area, dorsal part, layer 1"
ACA,3,CCF_2017,"Anterior cingulate area, dorsal part, layer 2/3"
ACA,4,CCF_2017,"Anterior cingulate area, dorsal part, layer 5"


In [69]:
lab.Person()

username,fullname
daiek,Kayvon Daie
daveliu,Dave Liu
HH,Han Hou
Kenta,Kenta Hagihara
liangy10,Yajie Liang
Lucas,Lucas Kinsey
NT,Nai-Wen Tien
rozsam,Marton Rozsa
singha,Amrita Singh
Smrithi,Smrithi Sunil


In [70]:
lab.Person.insert1({'username': 'LucasK', 'fullname': 'Lucas Kinsey'})

In [71]:
lab.Person()

username,fullname
daiek,Kayvon Daie
daveliu,Dave Liu
HH,Han Hou
Kenta,Kenta Hagihara
liangy10,Yajie Liang
Lucas,Lucas Kinsey
LucasK,Lucas Kinsey
NT,Nai-Wen Tien
rozsam,Marton Rozsa
singha,Amrita Singh
